In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import re
#re.compile('<title>(.*)</title>')
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

In [2]:
  #pip install --ignore-installed torch --user

In [3]:
df = pd.read_excel('C:/Users/SPEECH AND LANG GROU/Documents/condata.xlsx')
df

,Question,Answer
0,"Ẹ káàbọ̀, sà. Kí ni mo lè ṣe fún yín?",Mo fẹ́ẹ́ ra àkàrà-òyìnbó fún ọjọ́-ìbí ni
1,Ó dáa. Onírú èròja lẹ fẹ́ẹ́ rà?,Onípele méjì ti ó ní èròjà ‘strawberry’ ni
2,Ṣé àkàrà-òyìnbó aláìlẹ́yin lẹ fẹ́?,"Bẹ́ẹ̀ni, aláìlẹ́yin ni mo fẹ́."
3,"Ó dáa. Ṣùgbọn, ẹni tó bá béèrè ní a máa ń ṣe a...","Kò burú. Màá padà wá títí aago méjì, ṣùgbọ́n è..."
4,Báwo ni kí àkàrà-òyìnbó náà tóbi tó?,Oní-kílò mẹ́ta.
...,...,...
204,"Ó dáa, ẹ pò ó!",Ó dáa.
205,Eélòó ni àpò ilá?,Ọgọ́rùn-ún mẹ́fà náírà (N600) ni à ń ta àpò.
206,Ọgọ́rùn-ún mẹ́fà náírà (N600) ló bá dé láti ok...,Ẹ jẹ́ kí a san ogọ́rùn-ún márùn-ún àti àádọ́ta...
207,Àpò mélòó ni ẹ fẹ́?,Àpò kan péré ni.


In [4]:
#pip install torch

In [5]:
# total amount of rows and columns
total_rows=len(df.axes[0])
print("total rows", total_rows)
total_cols=len(df.axes[1])
print("total columns", total_cols)

total rows 209
total columns 2


In [6]:
#Case conversion
def lower(text):
    #convert all the text into lower letters
    #remove all special characters: except space ''
    text =text.lower()
    #text = re.sub('[^A-Za-z]+','',text)
    return text

In [7]:
df['Question'] = df['Question'].apply(lower)
df['Answer'] = df['Answer'].astype(str).apply(lower)

In [8]:
df.head(10)

,Question,Answer
0,"ẹ káàbọ̀, sà. kí ni mo lè ṣe fún yín?",mo fẹ́ẹ́ ra àkàrà-òyìnbó fún ọjọ́-ìbí ni
1,ó dáa. onírú èròja lẹ fẹ́ẹ́ rà?,onípele méjì ti ó ní èròjà ‘strawberry’ ni
2,ṣé àkàrà-òyìnbó aláìlẹ́yin lẹ fẹ́?,"bẹ́ẹ̀ni, aláìlẹ́yin ni mo fẹ́."
3,"ó dáa. ṣùgbọn, ẹni tó bá béèrè ní a máa ń ṣe a...","kò burú. màá padà wá títí aago méjì, ṣùgbọ́n è..."
4,báwo ni kí àkàrà-òyìnbó náà tóbi tó?,oní-kílò mẹ́ta.
5,á jẹ́ ẹgbẹ̀rún méje náírà (₦7000),ẹ̀dínwó ńkọ́?
6,"ẹ má bínú, ọ̀gá, j́alẹ̀jálẹ̀ ni mo ti sọ fún yín.","á-àá, èmi tí mo ti jẹ́ oníbàárà yín tipẹ́. ó y..."
7,"ẹ má bínú, ọ̀gá, ṣùgbọ́n n kò lè fún yíin ní ẹ...",ó dáa. màá fún ẹ ní ẹgbẹ̀rún méje náírà (₦700...
8,ẹ fọkànbalẹ̀. ẹ̀yin náà sáà mọ̀ wá. adùn ti wa...,ó dáa. màá wá ní aago méjì. mo fẹ́ kí ṣọnà òdò...
9,ìyẹn kò ṣòro. ẹ máa san ìdajì owó ní àsansílẹ̀.,"á-àá, ṣebí oníbàárà tipẹ́tipẹ́ ni mí. ẹ ò lè f..."


In [9]:
#Removal of special characters
def remove_special_characters (df, column, characters):
    for char in characters:
        df[column]= df[column].str.replace(char,'',regex=False)
    return df
#special characters to remove
special_characters = ['?','#','(',')',',','₦','!','-','N','']
#Remove special characters from columns
df = remove_special_characters(df, 'Question',special_characters)
df = remove_special_characters(df, 'Answer',special_characters)


In [10]:
df.head(10)

,Question,Answer
0,ẹ káàbọ̀ sà. kí ni mo lè ṣe fún yín,mo fẹ́ẹ́ ra àkàràòyìnbó fún ọjọ́ìbí ni
1,ó dáa. onírú èròja lẹ fẹ́ẹ́ rà,onípele méjì ti ó ní èròjà ‘strawberry’ ni
2,ṣé àkàràòyìnbó aláìlẹ́yin lẹ fẹ́,bẹ́ẹ̀ni aláìlẹ́yin ni mo fẹ́.
3,ó dáa. ṣùgbọn ẹni tó bá béèrè ní a máa ń ṣe al...,kò burú. màá padà wá títí aago méjì ṣùgbọ́n èl...
4,báwo ni kí àkàràòyìnbó náà tóbi tó,oníkílò mẹ́ta.
5,á jẹ́ ẹgbẹ̀rún méje náírà 7000,ẹ̀dínwó ńkọ́
6,ẹ má bínú ọ̀gá j́alẹ̀jálẹ̀ ni mo ti sọ fún yín.,áàá èmi tí mo ti jẹ́ oníbàárà yín tipẹ́. ó yẹ ...
7,ẹ má bínú ọ̀gá ṣùgbọ́n n kò lè fún yíin ní ẹ̀d...,ó dáa. màá fún ẹ ní ẹgbẹ̀rún méje náírà 7000 ...
8,ẹ fọkànbalẹ̀. ẹ̀yin náà sáà mọ̀ wá. adùn ti wa...,ó dáa. màá wá ní aago méjì. mo fẹ́ kí ṣọnà òdò...
9,ìyẹn kò ṣòro. ẹ máa san ìdajì owó ní àsansílẹ̀.,áàá ṣebí oníbàárà tipẹ́tipẹ́ ni mí. ẹ ò lè fọk...


In [11]:
#df['Answer'] = df['Answer'].astype(str).apply(preprocess)

In [12]:
#Tokenization using ByteLevelBPETokenizer

In [13]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, processors

In [14]:
all_texts = df['Question'].tolist() + df['Answer'].tolist()
#Initialize a byte pair encoding (BPE) Tokenizer
tokenizer =Tokenizer(models.BPE())
#Set the pre-tokenizer
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
#Initialize a trainer for the BPE model
trainer = trainers.BpeTrainer(special_tokens = ['<pad>','<s>','</s>','<unk>','<mask>'])
#Train the tokenizer on the dataset
tokenizer.train_from_iterator(all_texts,trainer)

#Tokenize the dataset
df['Question'] = df['Question'].apply(lambda x: tokenizer.encode(x).tokens)
df['Answer'] = df['Answer'].apply(lambda x: tokenizer.encode(x).tokens)

In [15]:
df

,Question,Answer
0,"[ẹ, káàbọ̀, sà, ., kí, ni, mo, lè, ṣe, fún, yín]","[mo, fẹ́ẹ́, ra, àkàràòyìnbó, fún, ọjọ́ìbí, ni]"
1,"[ó, dáa, ., onírú, èròja, lẹ, fẹ́ẹ́, rà]","[onípele, méjì, ti, ó, ní, èròjà, ‘, strawberr..."
2,"[ṣé, àkàràòyìnbó, aláìlẹ́yin, lẹ, fẹ́]","[bẹ́ẹ̀ni, aláìlẹ́yin, ni, mo, fẹ́, .]"
3,"[ó, dáa, ., ṣùgbọn, ẹni, tó, bá, béèrè, ní, a,...","[kò, burú, ., màá, padà, wá, títí, aago, méjì,..."
4,"[báwo, ni, kí, àkàràòyìnbó, náà, tóbi, tó]","[oníkílò, mẹ́ta, .]"
...,...,...
204,"[ó, dáa, ẹ, pò, ó]","[ó, dáa, .]"
205,"[eélòó, ni, àpò, ilá]","[ọgọ́rùnún, mẹ́fà, náírà, n600, ni, à, ń, ta, ..."
206,"[ọgọ́rùnún, mẹ́fà, náírà, n600, ló, bá, dé, lá...","[ẹ, jẹ́, kí, a, san, ogọ́rùnún, márùnún, àti, ..."
207,"[àpò, mélòó, ni, ẹ, fẹ́]","[àpò, kan, péré, ni, .]"


In [16]:
#Encoding using Byte Pair Encoding

In [17]:
def encode_text(text):
    return tokenizer.encode(text).ids

df['Question'] = df['Question'].astype(str).apply(encode_text)
df['Answer'] = df['Answer'].astype(str).apply(encode_text)

In [18]:
df

,Question,Answer
0,"[54, 820, 246, 5, 99, 62, 125, 174, 128, 97, 207]","[125, 403, 117, 348, 97, 916, 62]"
1,"[46, 90, 5, 788, 862, 278, 403, 64]","[791, 102, 77, 46, 74, 861, 56, 932, 57, 62]"
2,"[110, 348, 433, 278, 116]","[171, 433, 62, 125, 116, 5]"
3,"[46, 90, 5, 805, 719, 168, 124, 877, 74, 17, 1...","[130, 876, 5, 161, 291, 126, 520, 301, 102, 21..."
4,"[771, 62, 99, 348, 177, 799, 168]","[790, 251, 5]"
...,...,...
204,"[46, 90, 54, 216, 46]","[46, 90, 5]"
205,"[91, 62, 232, 616]","[92, 410, 69, 342, 62, 39, 49, 88, 232, 5]"
206,"[92, 410, 69, 342, 180, 124, 351, 402, 656, 21...","[54, 134, 99, 17, 148, 655, 162, 107, 113, 69,..."
207,"[232, 312, 62, 54, 116]","[232, 105, 294, 62, 5]"


In [19]:
# Vectorization using pytorch embedding layer
#Find the size of the vocabulary
vocab_size = tokenizer.get_vocab_size()


#Set the embedding dimension
embedding_dim = 50

#create an embedding layer
embedding = nn.Embedding(vocab_size, embedding_dim)

#function to convert token IDS to vectors
def tokens_to_vectors(token_ids):
    token_tensor = torch.tensor(token_ids)
    return embedding(token_tensor).detach().numpy()

#Convert token IDS to vectors for the dataset, then we have question vectors and answers vectors
df['Question'] = df['Question'].apply(tokens_to_vectors)
df['Answer'] = df['Answer'].apply(tokens_to_vectors)
df

,Question,Answer
0,"[[0.4790702, -0.47900626, 1.1731269, -1.149754...","[[-1.7355759, -1.4066396, -1.1472911, 0.024875..."
1,"[[0.3496512, 0.41629985, 0.3235675, -0.1275970...","[[1.5066491, 0.044439055, 0.858046, -1.5308855..."
2,"[[1.0629296, -1.2289413, -1.0061554, 0.5598995...","[[0.044735305, 0.34576195, 0.6047246, -1.24466..."
3,"[[0.3496512, 0.41629985, 0.3235675, -0.1275970...","[[0.3333585, 1.6317962, 0.70545495, -1.0731654..."
4,"[[0.2245794, 3.5018392, -1.0172527, 0.09436045...","[[-0.80014217, 0.4436188, 0.8936483, 0.3038669..."
...,...,...
204,"[[0.3496512, 0.41629985, 0.3235675, -0.1275970...","[[0.3496512, 0.41629985, 0.3235675, -0.1275970..."
205,"[[-0.7224956, 1.303537, -2.7278016, -0.0510044...","[[0.1563529, 0.7016215, -1.5666858, -1.3901995..."
206,"[[0.1563529, 0.7016215, -1.5666858, -1.3901995...","[[0.4790702, -0.47900626, 1.1731269, -1.149754..."
207,"[[-1.5406935, -0.58862615, -0.5522188, 1.20193...","[[-1.5406935, -0.58862615, -0.5522188, 1.20193..."


In [ ]:
#Dataset Split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df['Question'], df['Answer'], test_size=0.2
                                                   )

In [29]:
print("Number of Training Sample: ",X_train.shape)
print("Number of Training label: ",y_train.shape)
print("Number of Test Sample: ",X_test.shape)
print("Number of Test label: ",y_test.shape)

Number of Training Sample:  (167,)
Number of Training label:  (167,)
Number of Test Sample:  (42,)
Number of Test label:  (42,)


In [ ]:
#Model Training

In [ ]:
#Model Evaluation